In [339]:
import pandas as pd
import os, re
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine, types
from sqlalchemy import text 
from openpyxl import load_workbook

In [340]:
from dotenv import dotenv_values

config = dotenv_values()

# define variables for the login
pg_user = config['POSTGRES_USER']  # align the key label with your .env file !
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

In [341]:
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [342]:
my_schema = 'team_5' # update it to your schema

with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {my_schema};'))

In [343]:
df_1= pd.read_csv('../data/Raw/World_Marriage_Dataset.csv')

In [344]:
df_1.drop(columns=["Sr.No."], inplace=True)

In [345]:
df_1.columns = df_1.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [346]:
df_1.rename(columns={
    "agegroup": "age_group",
    "maritalstatus": "marital_status",
    "dataprocess": "data_process",
    "Data Collection (Start Year)": "data_collection_start_year",
    "Data Collection (End Year)": "data_collection_end_year",
    "Data Source": "data_source",
    "Country": "country",
    "Sex": "sex"
}, inplace=True)

In [347]:
df_1.drop_duplicates(inplace=True)

df_1['data_collection_start_year'] = df_1['data_collection_start_year'].astype(str).str.replace(',', '').astype(int)
df_1['data_collection_end_year'] = df_1['data_collection_end_year'].astype(str).str.replace(',', '').astype(int)

In [348]:
df_1.isnull().sum()

country                       0
age_group                     0
sex                           0
marital_status                0
data_process                  0
data_collection_start_year    0
data_collection_end_year      0
data_source                   0
dtype: int64

In [349]:
#df_1.to_csv("cleaned_world_marriage.csv", index=False)

In [350]:
#df_1.to_sql('world_marriage', engine, if_exists='replace', index=False)

In [351]:
df_2 = pd.read_csv('../data/Raw/age-at-marriage-women.csv')

In [352]:
df_2.columns = df_2.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [353]:
df_2 = df_2.drop(columns=['1005564annotations'])

df_2.rename(columns={
    "entity": "country",   
}, inplace=True)

In [354]:
df_2.drop_duplicates(inplace=True)


In [355]:
df_2['year'] = df_2['year'].astype(str).str.replace(',', '').astype(int)

In [356]:
df_2.isnull().sum()

country                                0
code                                   0
year                                   0
mean_age_of_women_at_first_marriage    0
dtype: int64

In [357]:
#df_2.to_csv("cleaned_age_at_marriage_women.csv", index=False)

In [358]:
#df_2.to_sql('age_at_marriage_women', engine, if_exists='replace', index=False)

In [359]:
df_3= pd.read_csv('../data/Raw/marriage-rate-per-1000-inhabitants.csv')

In [360]:
df_3.columns = df_3.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [361]:
df_3.rename(columns={
    "entity": "country",   
}, inplace=True)

In [362]:
df_3['year'] = df_3['year'].astype(str).str.replace(',', '').astype(int)

In [363]:
df_3.drop_duplicates(inplace=True)


In [364]:
df_3.isnull().sum()

country                                          0
code                                             0
year                                             0
crude_marriage_rate_marriages_per_1000_people    0
dtype: int64

In [365]:
#df_3.to_csv("cleaned_marriage-rate-per-1000-inhabitants.csv", index=False)

In [366]:
#df_3.to_sql('married_rate_per_1000', engine, if_exists='replace', index=False)

In [367]:
df_4= pd.read_csv('../data/Raw/marriage-rates-in-1990-vs-2020.csv')

In [368]:
df_4.columns = df_4.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [369]:
df_4 = df_4.drop(columns=['worldregionsaccordingtoowid'])

df_4.rename(columns={
    "crudemarriageratemarriagesper1000people": "crude_marriage_rate",
    "crudemarriageratemarriagesper1000people1": "crude_marriage_rate_people1",
    "year1": "year_1",
    "entity": "country"
}, inplace=True)

In [370]:
df_4.drop_duplicates(inplace=True)
df_4.dropna(inplace=True)

In [371]:
df_4['year_1'] = pd.to_numeric(df_4['year_1'], errors='coerce').astype('Int64')

In [372]:
df_4.isnull().sum()

country                        0
code                           0
year                           0
crude_marriage_rate            0
crude_marriage_rate_people1    0
year_1                         0
dtype: int64

In [373]:
#df_4.to_csv("cleaned_marriage-rates-in-1990-vs-2020.csv", index=False)

In [374]:
#df_4.to_sql('marriage_rates_in_1990_vs_2020', engine, if_exists='replace', index=False)

In [375]:
df_5 = pd.read_csv('../data/Raw/share-of-births-outside-marriage.csv')

In [376]:
df_5.columns = df_5.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [377]:

df_5.rename(columns={
    "shareofbirthsoutsideofmarriageofallbirths": "share_of_births_outside_of_marriage",
    "entity": "country"
}, inplace=True)

df_5.drop_duplicates(inplace=True)

In [378]:
df_5.isnull().sum()

country                                0
code                                   0
year                                   0
share_of_births_outside_of_marriage    0
dtype: int64

In [379]:
#df_5.to_csv("cleaned_share-of-births-outside-marriage.csv", index=False)

In [380]:
#df_5.to_sql('share_of_births_outside_marriage', engine, if_exists='replace', index=False)

In [381]:
df_6 = pd.read_csv('../data/Raw/share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv')

In [382]:
df_6.columns = df_6.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

df_6.drop_duplicates(inplace=True)
df_6.sample(5)

,entity,code,year,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort
6,Men,NaN,23,21.4,26.8,38.1,26.2,10.5,3.9,2.1,0.8
31,Men,NaN,48,92.5,91.7,92.3,81.3,69.9,NaN,NaN,NaN
32,Men,NaN,49,92.7,91.8,92.3,81.5,70.3,NaN,NaN,NaN
23,Men,NaN,40,89.6,89.9,91.1,78.0,64.0,53.9,NaN,NaN
58,Women,NaN,41,82.9,90.7,95.1,85.7,72.7,62.0,NaN,NaN


In [383]:
df_6 = df_6.drop(columns=['code','proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort'])

df_6.rename(columns={
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort": "1900_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort": "1920_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort": "1940_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort": "1960_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort": "1970_birthcohort",
    "entity": "sex"
}, inplace=True)

In [384]:
df_6.isnull().sum()

sex                 0
year                0
1900_birthcohort    0
1920_birthcohort    0
1940_birthcohort    0
1960_birthcohort    0
1970_birthcohort    0
dtype: int64

In [385]:
#df_6.to_csv("cleaned_share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv", index=False)

In [386]:
#df_6.to_sql('men_in_england_and_wales_married_by_age', engine, if_exists='replace', index=False)

In [387]:
df_7 = pd.read_csv('../data/Raw/share-of-births-outside-marriage.csv')

In [388]:
df_7.columns = df_7.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [389]:
df_7.rename(columns={
    "shareofsingleparenthouseholds": "share_of_single_parent_households",
    "entity": "country"
}, inplace=True)

df_7.drop_duplicates(inplace=True)
df_7.sample(5)

,country,code,year,shareofbirthsoutsideofmarriageofallbirths
2092,United States,USA,1974,13.2
1561,Poland,POL,2011,21.2
1468,New Zealand,NZL,2015,46.7
1707,Slovakia,SVK,2007,28.8
1250,Luxembourg,LUX,2009,32.1


In [390]:
df_7.isnull().sum()

country                                      0
code                                         0
year                                         0
shareofbirthsoutsideofmarriageofallbirths    0
dtype: int64

In [391]:
#df_7.to_csv("cleaned_share-of-single-parent-households.csv", index=False)

In [392]:
#df_7.to_sql('single_parent_households', engine, if_exists='replace', index=False)

In [393]:
df_8 = pd.read_csv('../data/Raw/share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv')

In [394]:
df_8.columns = df_8.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [395]:
df_8['code'] = df_8['code'].fillna('GBR')
df_8.sample(5)

,entity,code,year,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort
30,Men,GBR,47,92.3,91.6,92.2,81.0,69.4,NaN,NaN,NaN
2,Men,GBR,19,0.8,0.6,2.0,2.5,0.7,0.3,0.1,0.0
26,Men,GBR,43,91.1,90.8,91.7,79.5,66.7,56.6,NaN,NaN
14,Men,GBR,31,76.5,79.7,85.1,66.6,45.1,29.6,21.3,NaN
54,Women,GBR,37,80.6,89.4,94.4,84.1,69.4,57.8,NaN,NaN


In [396]:
df_8 = df_8.drop(columns=['code','proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort'])

df_8.rename(columns={
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort": "1900_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort": "1920_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort": "1940_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort": "1960_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort": "1970_birthcohort",
    "entity": "sex"
}, inplace=True)

df_8.drop_duplicates(inplace=True)
df_8.sample(5)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
41,Women,24,40.8,56.1,75.5,55.1,29.7
60,Women,43,83.7,91.1,95.3,86.2,73.7
1,Men,18,0.1,0.1,0.4,0.6,0.1
0,Men,17,0.0,0.0,0.0,0.1,0.0
4,Men,21,6.1,7.4,13.6,11.9,3.9


In [397]:
df_8.isnull().sum()

sex                 0
year                0
1900_birthcohort    0
1920_birthcohort    0
1940_birthcohort    0
1960_birthcohort    0
1970_birthcohort    0
dtype: int64

In [398]:
#df_8.to_csv("cleaned_share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv", index=False)

In [399]:
#df_8.to_sql('women_in_england_and_wales_married_by_age', engine, if_exists='replace', index=False)

In [400]:
#pip install openpyxl pywin32

In [401]:
df_excel_1 = pd.read_excel('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx')

In [402]:
#all_sheets = pd.read_excel('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx', sheet_name=None)

In [403]:
xls_1 = pd.ExcelFile('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx')
print(xls_1.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'MARITAL_STATUS_BY_AGE', 'CURRENTLY MARRIED', 'EVER_MARRIED', 'SMAM']


In [404]:
excel_1 = '../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx'

# Output directory (make sure it exists)
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

# List of sheets you want to extract
sheets_to_extract = ['MARITAL_STATUS_BY_AGE', 'CURRENTLY MARRIED', 'EVER_MARRIED', 'SMAM']

In [405]:
"""for sheet in sheets_to_extract:
    # Read just this sheet into a DataFrame
    df_excel_1 = pd.read_excel(excel_1, sheet_name=sheet)
    
    # Optional: Clean the filename (replace spaces with underscores, etc.)
    csv_name = sheet.replace(' ', '_').lower() + '.csv'
    csv_path = os.path.join(output_dir, csv_name)
    
    # Save the DataFrame as CSV
    df_excel_1.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")
"""

'for sheet in sheets_to_extract:\n    # Read just this sheet into a DataFrame\n    df_excel_1 = pd.read_excel(excel_1, sheet_name=sheet)\n    \n    # Optional: Clean the filename (replace spaces with underscores, etc.)\n    csv_name = sheet.replace(\' \', \'_\').lower() + \'.csv\'\n    csv_path = os.path.join(output_dir, csv_name)\n    \n    # Save the DataFrame as CSV\n    df_excel_1.to_csv(csv_path, index=False)\n    print(f"Saved: {csv_path}")\n'

In [406]:
xls_2 = pd.ExcelFile('../data/Raw/undesa_pd_2019_world_fertility_dataset.xlsx')
print(xls_2.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'FERTILITY INDICATORS']


In [407]:
excel_2 = '../data/Raw/undesa_pd_2019_world_fertility_dataset.xlsx'
sheet_name = 'FERTILITY INDICATORS'
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

df_excel_2 = pd.read_excel(excel_2, sheet_name=sheet_name)


In [408]:
"""csv_name = sheet_name.replace(' ', '_').lower() + '.csv'
csv_path = os.path.join(output_dir, csv_name)
df_excel_2.to_csv(csv_path, index=False)
print(f"Saved: {csv_path}")
"""

'csv_name = sheet_name.replace(\' \', \'_\').lower() + \'.csv\'\ncsv_path = os.path.join(output_dir, csv_name)\ndf_excel_2.to_csv(csv_path, index=False)\nprint(f"Saved: {csv_path}")\n'

In [409]:
xls_3 = pd.ExcelFile('../data/Raw/undesa_pd_ds_1970-2030_fp_rev-2024_rev.xlsx')
print(xls_3.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'Countries', 'Regions']


In [410]:
excel_3 = '../data/Raw/undesa_pd_ds_1970-2030_fp_rev-2024_rev.xlsx'
sheets_to_extract = ['Countries', 'Regions']
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)


In [411]:
"""
for sheet in sheets_to_extract:
    df = pd.read_excel(excel_3, sheet_name=sheet)
    csv_name = sheet.replace(' ', '_').lower() + '.csv'
    csv_path = os.path.join(output_dir, csv_name)
    df.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")

"""

'\nfor sheet in sheets_to_extract:\n    df = pd.read_excel(excel_3, sheet_name=sheet)\n    csv_name = sheet.replace(\' \', \'_\').lower() + \'.csv\'\n    csv_path = os.path.join(output_dir, csv_name)\n    df.to_csv(csv_path, index=False)\n    print(f"Saved: {csv_path}")\n\n'

In [412]:
df_9 = pd.read_csv('../data/Raw/unpopulation_dataportal_20250728095844.csv')
df_9.sample(5)

,IndicatorId,IndicatorName,IndicatorShortName,Source,SourceYear,Author,LocationId,Location,Iso2,Iso3,...,AgeStart,AgeEnd,Age,CategoryId,Category,EstimateTypeId,EstimateType,EstimateMethodId,EstimateMethod,Value
2838,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,96,Brunei Darussalam,BN,BRN,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,58.82
2409,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,76,Brazil,BR,BRA,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,60.84
19256,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,666,Saint Pierre and Miquelon,PM,SPM,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,3,Projection,62.08
13896,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,474,Martinique,MQ,MTQ,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,45.64
2612,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,90,Solomon Islands,SB,SLB,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,63.49


In [413]:
df_9.columns = df_9.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_9.sample(5)

,indicatorid,indicatorname,indicatorshortname,source,sourceyear,author,locationid,location,iso2,iso3,...,agestart,ageend,age,categoryid,category,estimatetypeid,estimatetype,estimatemethodid,estimatemethod,value
3178,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,108,Burundi,BI,BDI,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,60.44
15589,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,531,Curaçao,CW,CUW,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,42.51
20654,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,706,Somalia,SO,SOM,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,65.27
10649,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,376,Israel,IL,ISR,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,64.37
9165,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,316,Guam,GU,GUM,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,42.91


In [414]:
df_9 = df_9.drop(columns=['indicatorid','indicatorshortname',
    'source',
    'author', 'locationid', 'iso2','estimatetypeid','category','categoryid','agestart','ageend','author','ageid', 'estimatetype','variantid','sexid','timeid'])

df_9.rename(columns={
    "sourceyear": "year",
    "location": "country",
    "estimatemethodid": "estimate_method",
    "iso3": "code",
}, inplace=True)



In [415]:
df_9.drop_duplicates(inplace=True)

In [416]:
df_9

,indicatorname,year,country,code,time,variant,sex,age,estimate_method,estimatemethod,value
0,Currently married (Percent),2024,Afghanistan,AFG,1970,Median,Female,15-49,2,Interpolation,80.94
2,Currently married (Percent),2024,Afghanistan,AFG,1971,Median,Female,15-49,2,Interpolation,80.90
4,Currently married (Percent),2024,Afghanistan,AFG,1972,Median,Female,15-49,2,Interpolation,80.87
6,Currently married (Percent),2024,Afghanistan,AFG,1973,Median,Female,15-49,2,Interpolation,80.84
8,Currently married (Percent),2024,Afghanistan,AFG,1974,Median,Female,15-49,2,Interpolation,80.53
...,...,...,...,...,...,...,...,...,...,...,...
25078,Currently married (Percent),2024,Zambia,ZMB,2021,Median,Female,15-49,3,Projection,54.31
25080,Currently married (Percent),2024,Zambia,ZMB,2022,Median,Female,15-49,3,Projection,53.82
25082,Currently married (Percent),2024,Zambia,ZMB,2023,Median,Female,15-49,3,Projection,53.35
25084,Currently married (Percent),2024,Zambia,ZMB,2024,Median,Female,15-49,3,Projection,52.91


In [417]:
df_9.isnull().sum()

indicatorname      0
year               0
country            0
code               0
time               0
variant            0
sex                0
age                0
estimate_method    0
estimatemethod     0
value              0
dtype: int64

In [418]:
#df_9.to_csv("cleaned_unpopulation_dataportal.csv", index=False)

In [419]:
#df_9.to_sql('unpopulation_dataportal', engine, if_exists='replace', index=False)

In [420]:
df_10 = pd.read_csv('../data/processed/countries_un.csv',  header=5, low_memory=False)

In [421]:
df_10.columns = (
    df_10.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
)
df_10.sample(10)

,countryorarea,isocode,indicator,year,agegroup,percentage,number,dataprocess
129314,Togo,768,Married or in-union women,2015,25-29,81.563637,242.350850,Estimate
1654,Algeria,12,Married or in-union women,2014,45-49,78.610945,831.876346,Estimate
94471,Nicaragua,558,Married or in-union women,2033,15-49,55.361640,1151.693187,Projection
104014,Portugal,620,Married or in-union women,2011,45-49,76.670593,308.455764,Estimate
140654,Burkina Faso,854,Married or in-union women,1974,45-49,85.820000,105.743542,Estimate
58360,Iceland,352,Married or in-union women,1975,15-19,4.688571,0.511453,Estimate
78067,Malawi,454,Married or in-union women,2008,30-34,81.559130,310.636707,Estimate
131901,United Arab Emirates,784,Married or in-union women,2014,40-44,92.961434,217.310831,Estimate
39033,Eritrea,232,Married or in-union women,1989,20-24,69.400000,61.818744,Estimate
124476,Suriname,740,Married or in-union women,1977,35-39,76.092500,6.781744,Estimate


In [422]:
df_10.rename(columns={
    "dataprocess": "data_process",
}, inplace=True)

df_10.drop_duplicates(inplace=True)
df_10.sample(5)

,countryorarea,isocode,indicator,year,agegroup,percentage,number,data_process
102628,Philippines,608,Married or in-union women,2000,35-39,86.077143,2245.595566,Estimate
64556,Jamaica,388,Married or in-union women,2020,35-39,46.571429,52.161164,Estimate
92525,New Caledonia,540,Married or in-union women,2033,40-44,71.056828,7.743063,Projection
88900,Nauru,520,Married or in-union women,1985,35-39,88.100000,0.277075,Estimate
102637,Philippines,608,Married or in-union women,2001,40-44,85.025000,1907.300781,Estimate


In [423]:
for col in ['percentage', 'number']:
    if col in df_10.columns:
        df_10[col] = (
            df_10[col]
            .astype(str)
            .str.replace(',', '.', regex=False)
            .str.extract(r'([-+]?[0-9]*\.?[0-9]+)', expand=False)
            .astype(float)
            .round(2)
        )

In [424]:
unnamed_cols = [col for col in df_10.columns if 'unnamed' in col.lower()]
df_10.drop(columns=unnamed_cols, inplace=True)

In [425]:
df_10.dropna(inplace=True)

In [426]:
df_10.isnull().sum()

countryorarea    0
isocode          0
indicator        0
year             0
agegroup         0
percentage       0
number           0
data_process     0
dtype: int64

In [427]:
df_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145800 entries, 0 to 145799
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   countryorarea  145800 non-null  object 
 1   isocode        145800 non-null  int64  
 2   indicator      145800 non-null  object 
 3   year           145800 non-null  int64  
 4   agegroup       145800 non-null  object 
 5   percentage     145800 non-null  float64
 6   number         145800 non-null  float64
 7   data_process   145800 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 8.9+ MB


In [428]:
#df_10.to_csv("cleaned_countries_1970_2025_un.csv", index=False)

In [429]:
#df_10.to_sql('countries_1970_2025_un', engine, if_exists='replace', index=False)

In [430]:
df_11 = pd.read_csv('../data/processed/currently_married_un.csv',  header=2, low_memory=False)

In [431]:
df_11.sample(8)

,Country or area,ISO code,YearStart,YearEnd,Sex,AgeGroup,AgeStart,AgeEnd,DataValue,DataProcess,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Data,Note on Country and Population
34878,Nicaragua,558,2005,2005,Men,[75+],75,999,57.72,Census,2005 Census,1189,Nicaragua 2005 Census,UNSD,1.0,NaN,NaN
34285,Netherlands,528,2015,2015,Women,[60-64],60,64,69.23,Estimate,2015 Estimate,2170,Netherlands 2015 Estimate,UNSD,1.0,NaN,NaN
26038,Kazakhstan,398,2006,2006,Women,[30-34],30,34,73.66,Survey,2006 MICS,1882,Kazakhstan 2006 Multiple Indicator Cluster Survey,MICS,1.0,NaN,NaN
1684,Austria,40,1974,1974,Men,[25-29],25,29,60.47,Estimate,1974 Estimate,2038,Austria 1974 Estimate,UNSD,NaN,NaN,NaN
7927,China,156,2000,2000,Women,[45-49],45,49,95.74,Census,2000 Census,272,China 2000 Census,UNSD,1.0,NaN,NaN
20036,Hungary,348,1980,1980,Men,[30-34],30,34,83.36,Census,1980 Census,1092,Hungary 1980 Census,UNSD,NaN,NaN,NaN
25985,Kazakhstan,398,1989,1989,Women,[60-64],60,64,52.95,Census,1989 Census,1003,Kazakhstan 1989 Census,UNSD,NaN,NaN,NaN
3385,Belize,84,1970,1970,Men,[45-49],45,49,65.41,Census,1970 Census,940,Belize 1970 Census,US Census Bureau,NaN,NaN,NaN


In [432]:
df_11.columns = (
    df_11.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
    )
df_11.sample(8)

,countryorarea,isocode,yearstart,yearend,sex,agegroup,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteondata,noteoncountryandpopulation
2463,Bahamas,44,2000,2000,Men,[30-34],30,34,54.87,Census,2000 Census,415,Bahamas 2000 Census,UNSD,1.0,NaN,NaN
29609,Luxembourg,442,2011,2011,Women,[60-64],60,64,70.59,Census,2011 Census,4832,Luxembourg 2011 Census,Eurostat,1.0,Estimates computed based on data on marital st...,NaN
35612,Norway,578,1975,1975,Men,[55-59],55,59,82.08,Estimate,1975 Estimate,2180,Norway 1975 Estimate,UNSD,NaN,NaN,NaN
23466,Ireland,372,1983,1983,Men,[25-29],25,29,51.81,Estimate,1983 Estimate,2126,Ireland 1983 Estimate,UNSD,NaN,Based on the results of a labour force survey.,NaN
9800,Côte d'Ivoire,384,1994,1994,Men,[35-39],35,39,81.80,Survey,1994 DHS,1779,Cote d'Ivoire 1994 Demographic and Health Survey,DHS_STATcompiler,NaN,NaN,NaN
6163,Cameroon,120,1998,1998,Women,[15-19],15,19,33.60,Survey,1998 DHS,1674,Cameroon 1998 Demographic and Health Survey,DHS_STATcompiler,1.0,NaN,NaN
39385,Republic of Moldova,498,2004,2004,Men,[40-44],40,44,86.60,Census,2004 Census,845,Moldova 2004 Census,UNSD,1.0,NaN,NaN
20825,Hungary,348,2006,2006,Men,[55-59],55,59,74.68,Estimate,2006 Estimate,2120,Hungary 2006 Estimate,UNSD,NaN,NaN,NaN


In [433]:
df_11 = df_11.drop(columns = ['datacataloglongname', 'datacatalogid', 'yearstart' , 'yearend', 'noteondata', 'noteoncountryandpopulation', 'including_consensual_unions'])

df_11.rename(columns={
    "agestart": "age_start",
    "countryorarea": "country",
    "datasource": "data_source",
    "datavalue" : "data_value"
}, inplace=True)

df_11.sample(10)

,country,isocode,sex,agegroup,age_start,ageend,data_value,dataprocess,datacatalogshortname,data_source
2416,Bahamas,44,Men,[70-74],70,74,69.85,Census,1980 Census,UNSD
17291,Germany,276,Men,[55-59],55,59,71.72,Estimate,2014 Estimate,UNSD
34534,New Caledonia,540,Men,[40-44],40,44,44.45,Census,2014 Census,National statistics
42463,Senegal,686,Women,[35-39],35,39,93.02,Survey,1960-1961 ED,INED
42588,Senegal,686,Men,[40-44],40,44,91.70,Survey,1997 DHS,DHS_STATcompiler
11450,Denmark,208,Men,[75+],75,999,62.53,Census,1991 Census,UNSD
50357,Turks and Caicos Islands,796,Men,[35-39],35,39,77.66,Census,1970 Census,US Census Bureau
46965,Sweden,752,Men,[65-69],65,69,71.61,Estimate,1994 Estimate,UNSD
5177,Burkina Faso,854,Men,[75+],75,999,81.21,Census,1985 Census,UNSD
28769,Liechtenstein,438,Men,[65-69],65,69,79.12,Census,2010 Census,UNSD


In [434]:
df_11.drop_duplicates(inplace=True)

In [435]:
df_11.isnull().sum()

country                 0
isocode                 0
sex                     0
agegroup                0
age_start               0
ageend                  0
data_value              0
dataprocess             0
datacatalogshortname    0
data_source             0
dtype: int64

In [436]:
#df_11.to_csv("cleaned_currently_married_un.csv", index=False)

In [437]:
#df_11.to_sql('currently_married_un', engine, if_exists='replace', index=False)

In [438]:
df_12 = pd.read_csv('../data/processed/ever_married_un.csv', header= 2, low_memory = False)
df_12.head()

,Country or area,ISO code,YearStart,YearEnd,Sex,AgeGroup,AgeStart,AgeEnd,DataValue,DataProcess,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Data,Note on Country and Population
0,Afghanistan,4,1972,1974,Men,[15-19],15,19,7.7,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
1,Afghanistan,4,1972,1974,Men,[20-24],20,24,32.6,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
2,Afghanistan,4,1972,1974,Men,[25-29],25,29,61.4,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
3,Afghanistan,4,1972,1974,Men,[30-34],30,34,83.0,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
4,Afghanistan,4,1972,1974,Men,[35-39],35,39,91.2,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN


In [439]:
df_12.columns = (
    df_12.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
    )
df_12.sample(8)

,countryorarea,isocode,yearstart,yearend,sex,agegroup,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteondata,noteoncountryandpopulation
33231,Mexico,484,2016,2016,Men,[75+],75,999,96.26,Estimate,2016 Estimate,2158,Mexico 2016 Estimate,UNSD,1.0,NaN,NaN
54949,Western Sahara,732,1970,1970,Men,[55-59],55,59,90.86,Census,1970 Census,1106,Western Sahara 1970 Census,UNSD,1.0,NaN,Comprises of Northern Region (former Saguia el...
37741,Norway,578,1979,1979,Women,[40-44],40,44,94.77,Estimate,1979 Estimate,2180,Norway 1979 Estimate,UNSD,NaN,NaN,NaN
42705,Saint Lucia,662,1991,1991,Men,[75+],75,999,85.19,Census,1991 Census,1136,Saint Lucia 1991 Census,UNSD,NaN,NaN,NaN
21525,Hungary,348,1985,1985,Women,[40-44],40,44,95.80,Estimate,1985 Estimate,2120,Hungary 1985 Estimate,UNSD,NaN,NaN,NaN
20534,Guyana,328,2000,2000,Women,[15-19],15,19,31.65,Survey,2000 MICS,1939,Guyana 2000 Multiple Indicator Cluster Survey,MICS,NaN,NaN,NaN
12054,Denmark,208,1979,1979,Men,[75+],75,999,92.59,Estimate,1979 Estimate,2081,Denmark 1979 Estimate,UNSD,NaN,NaN,Excluding Faeroe Islands and Greenland shown s...
3107,Belgium,56,1970,1970,Men,[15-19],15,19,1.03,Census,1970 Census,290,Belgium 1970 Census,UNSD,NaN,NaN,NaN


In [440]:
df_12 = df_12.drop(columns = ['yearstart', 'yearend', 'datacatalogshortname', 'datacatalogid', 'datacataloglongname', 'including_consensual_unions', 'noteondata', 'noteoncountryandpopulation'])

df_12.rename(columns={
    "agestart": "age_start",
    "ageend": "age_end",
    "countryorarea": "country"
}, inplace=True)
df_12.sample(8)

,country,isocode,sex,agegroup,age_start,age_end,datavalue,dataprocess,datasource
47998,Sudan,729,Women,[45-49],45,49,96.08,Estimate,UNSD
53346,United Kingdom,826,Men,[10-14],10,14,0.00,Estimate,UNSD
8385,China,156,Women,[65+],65,999,99.72,Survey,National statistics
51548,Tonga,776,Men,[65-69],65,69,94.32,Census,UNSD
24647,Iraq,368,Women,[35-39],35,39,87.21,Survey,MICS
46324,Slovenia,705,Women,[35-39],35,39,72.46,Estimate,UNSD
25209,Ireland,372,Women,[15-19],15,19,0.37,Estimate,UNSD
47029,South Africa,710,Men,[65-69],65,69,94.70,Census,UNSD


In [441]:
df_12.dropna(inplace=True)

In [442]:
df_12.isnull().sum()

country        0
isocode        0
sex            0
agegroup       0
age_start      0
age_end        0
datavalue      0
dataprocess    0
datasource     0
dtype: int64

In [443]:
#df_12.to_csv("cleaned_ever_married_un.csv", index=False)

In [444]:
#df_12.to_sql('ever_married_un', engine, if_exists= 'replace', index= False)

In [445]:
df_13 = pd.read_csv('../data/processed/fertility_indicators_un.csv', header=6, low_memory=False)
df_13.head()

,Country or Area,Country or Area Code,Age Group,Indicator,Date,Value,Series,DataType,Data Source Type,Survey Programme,Data Source Inventory ID,Data Source Name,Data Source Name (short),Data Source Start Year,Data Source End Year,Reference,Reference Year
0,Afghanistan,4,[Total],TFR,1964.977051,7.966653,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
1,Afghanistan,4,[Total],TFR,1965.977051,8.212275,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
2,Afghanistan,4,[Total],TFR,1966.977051,8.317603,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
3,Afghanistan,4,[Total],TFR,1967.977051,8.225812,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
4,Afghanistan,4,[Total],TFR,1968.977051,8.068459,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012


In [446]:
df_13.columns = (df_13.columns
        .str.lower()
        .str.strip()
        .str.replace(' ', '')
        .str.replace('(', '')
        .str.replace(')', '')
        .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
        )

df_13.sample(6)

,countryorarea,countryorareacode,agegroup,indicator,date,value,series,datatype,datasourcetype,surveyprogramme,datasourceinventoryid,datasourcename,datasourcenameshort,datasourcestartyear,datasourceendyear,reference,referenceyear
53686,Norway,578,[20-24],ASFR2024,1995.500000,77.490010,Eurostat.20190531,Official estimates,Estimate,Estimate,2180,All sources of estimates,Estimates,1995,1995,"Eurostat Statistics, Fertility rates by age [d...",2019
30235,Guam,316,[20-24],ASFR2024,2010.500000,148.800000,NVSR.61.1,Direct,Register,VR,589,Vital Registration,Register,2010,2010,"National Vital Statistics Reports. Vol61, N1 (...",2012
54975,Paraguay,600,[Total],TFR,1986.869995,4.655122,"2004 ENDSSR-RHS,Birth Histories (Extrapolated)...",Extrapolated from Truncated Birth Histories,Survey,RHS,649,Paraguay 2004 Encuesta Nacional de Demografía ...,2004 ENDSSR-RHS,2004,2004,DHS Statcompiler,2012
48489,Morocco,504,[25-29],ASFR2529,1978.500000,281.831000,"1980 WFS,Birth Histories,FBH analysis 2018,771...",Birth histories,Survey,WFS,771,Morocco 1980 World Fertility Survey,1980 WFS,1979,1980,Fertility rates from full birth histories anal...,2018
44802,Malaysia,458,[Total],TFR,2010.501370,2.100000,"Register, VS Malaysia",Direct,Register,VR,616,Vital Registration,Register,2010,2010,Vital Statistics Malaysia,2017
12664,Canada,124,[Total],MAC,1992.500000,27.949500,"Register,Computed rate from DYB,DYB,427-135-48",Computed rate from DYB,Register,VR,427,Vital Registration,Register,1992,1992,Demographic Yearbook,1995


In [447]:
df_13 = df_13.drop(columns=['countryorareacode','indicator','datasourceinventoryid','surveyprogramme','series','datasourcename','reference','referenceyear'])

df_13.replace({
    "agegroup": "age_group",
    "countryorarea": "country",
    "datatype": "data_type",
},inplace=True)

In [448]:
df_13['date'] = df_13['date'].astype(int)
df_13['value'] = df_13['value'].round(2)
df_13.sample(12)

,countryorarea,agegroup,date,value,datatype,datasourcetype,datasourcenameshort,datasourcestartyear,datasourceendyear
50462,Nepal,[20-24],1993,289.00,Direct,Survey,2001 DHS,2001,2001
24897,Finland,[35-39],1979,27.54,Official estimates,Estimate,Estimates,1979,1979
20333,Dem. People's Rep. of Korea,[Total],2000,1.87,Reverse survival method,Survey,2009 MICS,2009,2009
60463,Rwanda,[20-24],1991,227.00,Direct,Survey,1992 DHS,1992,1992
6822,Belarus,[15-19],1988,36.00,Fertility data (adjusted),Estimate,Estimates,1988,1988
49606,Namibia,[20-24],1996,172.67,Birth histories,Survey,2013 DHS,2013,2013
43632,Luxembourg,[30-34],1994,97.97,Official estimates,Estimate,Estimates,1994,1994
49146,Myanmar,[35-39],1982,138.00,Own-children method,Survey,1991 PCFS,1991,1991
50327,Nepal,[35-39],1988,122.00,Own-children method,Survey,1995-1996 LSMS,1995,1996
69070,Tajikistan,[Total],1996,4.28,Reverse survival method,Survey,2000 MICS,2000,2000


In [449]:
#df_13.to_csv("cleaned_fertility_indicators.csv", index=False)

In [450]:
#df_13.to_sql('fertility_indicators_un',engine, if_exists='replace', index=False)

In [451]:
df_14 = pd.read_csv('../data/processed/marital_status_by_age_un.csv', header= 2, low_memory=False)
df_14.head()

,Country or area,ISO code,YearStart,YearEnd,Sex,MaritalStatus,Non-standard_AgeGroups,Series_contains_Non-standard_AgeGroups,AgeGroup,AgeStart,...,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Age groups,Note on Marital Status,Note on Data,Note on Country and Population,Note Other
0,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[15-19],15,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[20-24],20,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[25-29],25,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[30-34],30,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[35-39],35,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN


In [452]:
df_14.columns= (df_14.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '' , regex=True)  
    )
df_14.sample(5)

,countryorarea,isocode,yearstart,yearend,sex,maritalstatus,nonstandard_agegroups,series_contains_nonstandard_agegroups,agegroup,agestart,...,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteonagegroups,noteonmaritalstatus,noteondata,noteoncountryandpopulation,noteother
100734,Guyana,328,2002,2002,Men,Widowed,1.0,NaN,[55+],55,...,2002 Census,1225,Guyana 2002 Census,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
56635,Czechia,203,2006,2006,Women,Single,NaN,NaN,[15-19],15,...,2006 Estimate,2079,Czech Republic 2006 Estimate,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
111745,Iceland,352,1990,1990,Men,Widowed,NaN,NaN,[15-19],15,...,1990 Estimate,2121,Iceland 1990 Estimate,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
204690,Russian Federation,643,2010,2010,Women,Married,NaN,NaN,[35-39],35,...,2010 Census,4841,Russian Federation 2010 Census,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
82333,France,250,1979,1979,Women,Single,NaN,NaN,[75+],75,...,1979 Estimate,2094,France 1979 Estimate,UNSD,NaN,NaN,NaN,NaN,Excluding diplomatic personnel outside the cou...,NaN


In [453]:
df_14 = df_14.drop(columns=['datacataloglongname', 'noteondata', 'noteoncountryandpopulation','noteonagegroups', 'noteother',
                             'including_consensual_unions','isocode', 'datacatalogid', 'noteonmaritalstatus', 'series_contains_nonstandard_agegroups','nonstandard_agegroups'])

df_14.rename(columns={
    "countryorarea": "country",
    "agegroup": "age_group",
    "maritalstatus": "marital_status",
    "yearstart": "year_start",
    "yearend": "year_end",
    }, inplace =True
    )

df_14.sample(10)

,country,year_start,year_end,sex,marital_status,age_group,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datasource
69991,Egypt,2014,2014,Women,Divorced,[45-49],45,49,2.90,Survey,2014 DHS,DHS_STATcompiler
192067,Peru,2003,2006,Women,Never married,[20-24],20,24,55.90,Survey,2004-2006 DHS,DHS_STATcompiler
89340,Germany,2013,2013,Women,Single,[65-69],65,69,4.73,Estimate,2013 Estimate,UNSD
227992,Spain,1978,1978,Women,Widowed,[45-49],45,49,4.97,Estimate,1978 Estimate,UNSD
170732,Netherlands,1985,1985,Men,Divorced,[75+],75,999,2.16,Estimate,1985 Estimate,UNSD
170310,Netherlands,1981,1981,Men,Single,[65-69],65,69,6.25,Estimate,1981 Estimate,UNSD
170773,Netherlands,1985,1985,Men,Widowed,[70-74],70,74,11.34,Estimate,1985 Estimate,UNSD
124724,Israel,2002,2002,Women,Widowed,[65+],65,999,51.38,Estimate,2002 Estimate,UNSD
114847,Iceland,2016,2016,Men,Separated,[60-64],60,64,2.26,Estimate,2016 Estimate,UNSD
69193,Egypt,2000,2000,Women,Not living together,[50-54],50,54,0.60,Survey,2000 DHS,DHS_HH


In [454]:
df_14.drop_duplicates(inplace=True)
df_14.isnull().sum()

country                 0
year_start              0
year_end                0
sex                     0
marital_status          0
age_group               0
agestart                0
ageend                  0
datavalue               0
dataprocess             0
datacatalogshortname    0
datasource              0
dtype: int64

In [455]:
#df_14.to_csv("cleaned_marital_status_by_age_un.csv", index=False)

In [456]:
#df_14.to_sql('marital_status_by_age_un', engine, if_exists='replace', index=False)

In [457]:
df_15 = pd.read_csv('../data/processed/regions_un.csv', header=5, low_memory= False)
df_15.head(10)

,Region and subregion,ISO code,Regional Classification,Indicator,Year,AgeGroup,Percentage,Number,DataProcess
0,World,900,M49,Married or in-union women,1970,15-19,22.576683,7.186782e+04,Estimate
1,World,900,M49,Married or in-union women,1970,20-24,63.802057,1.628604e+05,Estimate
2,World,900,M49,Married or in-union women,1970,25-29,87.174827,1.826811e+05,Estimate
3,World,900,M49,Married or in-union women,1970,30-34,90.825027,1.791214e+05,Estimate
4,World,900,M49,Married or in-union women,1970,35-39,90.284386,1.615263e+05,Estimate
5,World,900,M49,Married or in-union women,1970,40-44,86.483531,1.393344e+05,Estimate
6,World,900,M49,Married or in-union women,1970,45-49,82.680237,1.160884e+05,Estimate
7,World,900,M49,Married or in-union women,1970,15-49,69.379111,1.013480e+06,Estimate
8,World,900,M49,Married or in-union women,1971,15-19,22.630416,7.412762e+04,Estimate
9,World,900,M49,Married or in-union women,1971,20-24,63.613178,1.700873e+05,Estimate


In [458]:
df_15.columns = (df_15.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(','')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
    )
df_15.sample(6)

,regionandsubregion,isocode,regionalclassification,indicator,year,agegroup,percentage,number,dataprocess
12637,Europe,908,SDG-M49,Married or in-union women,2010,40-44,76.070509,20398.824756,Estimate
23624,Developing countries,902,Development group,Married or in-union women,2007,15-19,15.227034,40421.086732,Estimate
23296,Developed countries,901,Development group,Married or in-union women,2047,15-19,4.022915,1177.695196,Projection
2032,Central and Southern Asia,62,SDG,Married or in-union women,1981,15-19,44.051122,44493.023964,Estimate
6135,Middle Africa,911,M49,Married or in-union women,2007,15-49,64.385284,36111.968909,Estimate
20998,Melanesia,928,SDG-M49,Married or in-union women,2002,45-49,84.680038,241.650577,Estimate


In [459]:
df_15 = df_15.drop(columns=['regionalclassification'])

df_15.rename(columns={
    "regionandsubregion": "region",
    "isocode": "iso_code",
    "agegroup": "age_group",
    "dataprocess": "process"
}, inplace=True)

df_15.sample(10)

,region,iso_code,indicator,year,age_group,percentage,number,process
6873,Northern Africa,912,Married or in-union women,2019,20-24,41.550468,8459.036327,Estimate
7117,Northern Africa,912,Married or in-union women,2049,40-44,80.600859,18626.113813,Projection
3803,Oceania excluding Australia and New Zealand,543,Married or in-union women,2040,30-34,77.667870,997.640682,Projection
890,Sub-Saharan Africa,202,Married or in-union women,2000,25-29,78.632949,38945.935539,Estimate
5606,Eastern Africa,910,Married or in-union women,2022,45-49,73.598118,12858.895407,Estimate
1527,Northern Africa and Western Asia,747,Married or in-union women,1998,15-49,61.250642,105350.164543,Estimate
568,World,900,Married or in-union women,2041,15-19,10.031292,58957.478892,Projection
9771,Eastern Asia,906,Married or in-union women,1976,30-34,97.081497,57392.704987,Estimate
5058,Africa,903,Married or in-union women,2035,25-29,72.068406,114435.034553,Projection
12484,Europe,908,Married or in-union women,1991,35-39,82.534949,22328.093970,Estimate


In [460]:
df_15.dropna(inplace=True)
df_15.isnull().sum()

region        0
iso_code      0
indicator     0
year          0
age_group     0
percentage    0
number        0
process       0
dtype: int64

In [461]:
print(df_15['number'] % 1 != 0)

0        True
1        True
2        True
3        True
4        True
         ... 
28507    True
28508    True
28509    True
28510    True
28511    True
Name: number, Length: 28512, dtype: bool


In [462]:
df_15['percentage'] = df_15['percentage'].round(2)
df_15['number'] = df_15['number'].astype(int)
df_15.head(10)

,region,iso_code,indicator,year,age_group,percentage,number,process
0,World,900,Married or in-union women,1970,15-19,22.58,71867,Estimate
1,World,900,Married or in-union women,1970,20-24,63.80,162860,Estimate
2,World,900,Married or in-union women,1970,25-29,87.17,182681,Estimate
3,World,900,Married or in-union women,1970,30-34,90.83,179121,Estimate
4,World,900,Married or in-union women,1970,35-39,90.28,161526,Estimate
5,World,900,Married or in-union women,1970,40-44,86.48,139334,Estimate
6,World,900,Married or in-union women,1970,45-49,82.68,116088,Estimate
7,World,900,Married or in-union women,1970,15-49,69.38,1013479,Estimate
8,World,900,Married or in-union women,1971,15-19,22.63,74127,Estimate
9,World,900,Married or in-union women,1971,20-24,63.61,170087,Estimate


In [463]:
#df_15.to_csv('cleaned_regions_un.csv', index=False)



In [464]:
#df_15.to_sql('regions_un', engine, if_exists='replace',index=False)

In [465]:
df_16_1 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa1.csv')
df_16_1
#Data for Chart SF1.1.A. Average size of households by household type, 2024a
# avg_size_all	avg_size_couple_with_children	avg_size_single_parent_with_children		

,Country,All households,Couple households with children,Single parent households with children
0,Mexico,"3,56","4,08","2,76"
1,Costa Rica,"3,46","4,37","3,44"
2,Türkiye,"3,20","4,10","2,80"
3,Israel,"3,19","4,65","2,86"
4,Columbia,"3,10",NaN,NaN
5,Slovak Republic,"3,10","3,80","2,50"
6,Chile,"2,80",NaN,NaN
7,Iceland,"2,70","4,12","2,61"
8,New Zealand,"2,61","3,88","2,67"
9,Greece,"2,60","3,80","2,50"


In [466]:
df_16_1.columns = df_16_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [467]:
df_16_1.rename(columns={
        "All households": "avg_size_all",
        "Couple with children": "avg_size_couple_with_children",
        "Single parent with children": "avg_size_single_parent_with_children"
}, inplace=True)

In [468]:
df_16_1.drop_duplicates(inplace=True)
df_16_1.dropna(inplace=True)

In [469]:
for col in df_16_1.columns:
    if col != 'country':
        # Replace commas with dots if necessary, remove non-numeric chars, convert to float
        df_16_1[col] = (
            df_16_1[col]
            .astype(str)  # ensure string for replace
            .str.replace(',', '.', regex=False)  # decimal commas to dots
            .str.replace(r'[^\d\.\-]', '', regex=True)  # remove non-numeric chars except dot and minus
            .replace('', None)  # empty to NaN
            .astype(float)  # convert to float
        )

# Check updated dtypes
print(df_16_1.dtypes)

country                                    object
all_households                            float64
couple_households_with_children           float64
single_parent_households_with_children    float64
dtype: object


In [470]:
info_16_1 = pd.DataFrame({
    'dtype': df_16_1.dtypes,
    'null_count': df_16_1.isnull().sum(),
    'unique_count': df_16_1.nunique()
})
print(info_16_1)

                                          dtype  null_count  unique_count
country                                  object           0            39
all_households                          float64           0            19
couple_households_with_children         float64           0            16
single_parent_households_with_children  float64           0            15


In [471]:
df_16_1

,country,all_households,couple_households_with_children,single_parent_households_with_children
0,Mexico,3.56,4.08,2.76
1,Costa Rica,3.46,4.37,3.44
2,Türkiye,3.20,4.10,2.80
3,Israel,3.19,4.65,2.86
5,Slovak Republic,3.10,3.80,2.50
7,Iceland,2.70,4.12,2.61
8,New Zealand,2.61,3.88,2.67
9,Greece,2.60,3.80,2.50
10,Croatia,2.60,3.90,2.60
11,Australia,2.53,3.93,2.78


In [472]:
df_16_2 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa2.csv', header=1)
df_16_2
#Table SF1.1.A. Types of household, 2021a
# share_couple_total	share_couple_with_children	share_couple_without_children	share_single_parent_total	share_single_mother	share_single_father	share_single_person	share_other_types						

,Country,Total,With children,Without children,Total.1,Single mother households,Single father households,Single person households,Other households types
0,Australia,"55,93","29,90","26,02","10,37",NaN,NaN,"25,12","8,58"
1,Austria,"48,93","21,13","27,80","5,63","4,78","0,85","38,34","7,11"
2,Belgium,"52,22","23,98","28,24","7,42","6,08","1,35","35,50","4,86"
3,Canada,"50,92","25,30","25,62","8,72",NaN,NaN,"29,35","11,02"
4,Chile,..,..,..,..,..,..,..,..
5,Columbia,..,..,..,..,..,..,..,..
6,Costa Rica,"52,44","38,15","14,29","10,55","9,49","1,06","11,27","25,74"
7,Czechia,"47,03","21,70","25,32","7,15","6,11","1,04","39,15","6,67"
8,Denmark,"48,60","20,41","28,19","6,31","5,11","1,19","37,57","7,52"
9,Estonia,"46,20","25,46","20,73","6,83","6,09","0,74","36,99","9,98"


In [473]:
df_16_2.rename(columns={
    "Total": "couple_total(%)",
    "Couple with children": "couple_with_children(%)",
    "Couple without children": "couple_without_children(%)",
    "Total.1": "single_parent_total(%)",
    "Single mother households": "single_mother(%)",
    "Single father households": "single_father(%)",
    "Single person households": "single_person(%)",
    "Other types of households": "other_household_types(%)"
}, inplace=True)

In [474]:
# Normalize column names (lowercase, underscores, remove () and non-ascii)
df_16_2.columns = (
    df_16_2.columns.str.strip().str.lower()
    .str.replace(' ', '_')
    .str.replace('[()%]', '', regex=True)
    .str.replace('[^0-9a-z_]', '', regex=True)
)

In [475]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_16_2.columns if c != "country"]

df_16_2[num_cols] = (
    df_16_2[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [476]:
df_16_2.drop_duplicates(inplace=True)
df_16_2.dropna(inplace=True)
df_16_2.dropna(how="all", subset=num_cols, inplace=True)

In [477]:
info_16_2 = pd.DataFrame({
    "dtype": df_16_2.dtypes,
    "null_count": df_16_2.isna().sum(),
    "unique_count": df_16_2.nunique()
})
print(info_16_2)
print(df_16_2.dtypes)

                          dtype  null_count  unique_count
country                  object           0            36
couple_total            float64           0            36
with_children           float64           0            35
without_children        float64           0            36
single_parent_total     float64           0            34
single_mother           float64           0            32
single_father           float64           0            31
single_person           float64           0            35
other_households_types  float64           0            36
country                    object
couple_total              float64
with_children             float64
without_children          float64
single_parent_total       float64
single_mother             float64
single_father             float64
single_person             float64
other_households_types    float64
dtype: object


In [478]:
df_16_2

,country,couple_total,with_children,without_children,single_parent_total,single_mother,single_father,single_person,other_households_types
1,Austria,48.93,21.13,27.80,5.63,4.78,0.85,38.34,7.11
2,Belgium,52.22,23.98,28.24,7.42,6.08,1.35,35.50,4.86
6,Costa Rica,52.44,38.15,14.29,10.55,9.49,1.06,11.27,25.74
7,Czechia,47.03,21.70,25.32,7.15,6.11,1.04,39.15,6.67
8,Denmark,48.60,20.41,28.19,6.31,5.11,1.19,37.57,7.52
9,Estonia,46.20,25.46,20.73,6.83,6.09,0.74,36.99,9.98
10,Finland,45.64,17.06,28.58,5.43,4.50,0.93,45.34,3.60
11,France,49.73,22.19,27.54,7.68,6.23,1.45,37.78,4.81
12,Germany,45.78,17.89,27.89,5.41,4.44,0.98,43.14,5.67
13,Greece,52.14,24.03,28.11,4.66,3.82,0.84,32.35,10.85


In [479]:
df_16_3 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa3.csv', header=1)
df_16_3
#Table SF1.1.B. Households by number of children, 2024a
# share_hh_0_children	share_hh_1_child	share_hh_2_children	share_hh_3plus_children		

,country,0 children,1 child,2 children,3 or more children,Children under 6
0,Australia,..,..,..,..,..
1,Austria,"77,78","10,52","8,57","3,12","9,44"
2,Belgium,"73,97","11,76","10,15","4,11","10,40"
3,Canada,..,..,..,..,..
4,Chile,..,..,..,..,..
5,Columbia,..,..,..,..,..
6,Costa Rica,"30,29","23,08","24,61","22,02","26,30"
7,Czechia,"71,95","13,85","11,56","2,64","12,29"
8,Denmark,"77,78","10,54","8,94","2,74","8,15"
9,Estonia,"75,76","12,53","8,73","2,98","9,85"


In [480]:
df_16_3.rename(columns={
    "0 children": "households_0_children(%)",
    "1 child": "households_1_child(%)",
    "2 children": "households_2_children(%)"
}, inplace=True)

In [481]:
# Normalize column names (lowercase, underscores, remove () and non-ascii)
df_16_3.columns = (
    df_16_3.columns.str.strip().str.lower()
    .str.replace(' ', '_')
    .str.replace('[()%]', '', regex=True)
    .str.replace('[^0-9a-z_]', '', regex=True)
)

In [482]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_16_3.columns if c != "country"]

df_16_3[num_cols] = (
    df_16_3[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [483]:
df_16_3.drop_duplicates(inplace=True)
df_16_3.dropna(inplace=True)

In [484]:
info_16_3 = pd.DataFrame({
    "dtype": df_16_3.dtypes,
    "null_count": df_16_3.isna().sum(),
    "unique_count": df_16_3.nunique()
})
print(info_16_3)
print(df_16_3.dtypes)

                         dtype  null_count  unique_count
country                 object           0            33
households_0_children  float64           0            32
households_1_child     float64           0            32
households_2_children  float64           0            33
3_or_more_children     float64           0            31
children_under_6       float64           0            31
country                   object
households_0_children    float64
households_1_child       float64
households_2_children    float64
3_or_more_children       float64
children_under_6         float64
dtype: object


In [485]:
df_16_3

,country,households_0_children,households_1_child,households_2_children,3_or_more_children,children_under_6
1,Austria,77.78,10.52,8.57,3.12,9.44
2,Belgium,73.97,11.76,10.15,4.11,10.40
6,Costa Rica,30.29,23.08,24.61,22.02,26.30
7,Czechia,71.95,13.85,11.56,2.64,12.29
8,Denmark,77.78,10.54,8.94,2.74,8.15
9,Estonia,75.76,12.53,8.73,2.98,9.85
10,Finland,81.98,7.89,6.99,3.14,7.14
11,France,75.36,11.43,9.23,3.99,9.86
12,Germany,79.86,9.91,7.72,2.51,8.57
13,Greece,74.31,11.83,9.97,3.89,9.37


In [486]:
df_17_1 = pd.read_csv('../data/Raw/OECD/SF_2_1_Fertility_rates_Births_by_birth_order_S2.csv')
df_17_1

,Country,Birth order,1987,1988,1989,1990,1991,1992,1993,1994,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Austria,First birth,"47,6","47,8","46,7","46,2","46,5","46,1","45,2","44,5",...,"48,0","48,3","47,3","47,5","47,1","47,2","47,7","47,6","48,4","48,1"
1,NaN,Second birth,"33,7","33,7","34,3","34,9","34,5","34,8","35,8","36,4",...,"35,5","35,3","35,6","35,3","35,3","35,1","35,3","35,5","34,9","35,1"
2,NaN,Third birth or higher,"18,8","18,5","19,0","18,9","19,0","19,1","18,9","19,1",...,"16,5","16,4","17,1","17,2","17,6","17,7","17,0","16,9","16,7","16,8"
3,Belgium,First birth,"46,8","46,9","47,3","47,3","48,1","47,2","46,9","47,2",...,"42,3","43,5","44,1","43,6","42,9","42,6","45,0","44,0","44,7","45,5"
4,NaN,Second birth,"33,0","32,9","32,7","32,8","32,3","32,8","33,5","33,0",...,"35,1","34,8","34,5","34,6","34,5","34,7","34,2","35,1","34,3","34,1"
5,NaN,Third birth or higher,"20,2","20,2","19,9","19,9","19,6","20,0","19,6","19,8",...,"22,6","21,8","21,4","21,9","22,6","22,6","20,8","20,9","20,9","20,4"
6,Czechia,First birth,"46,7","46,6","47,4","47,8","50,1","49,8","48,5","47,7",...,"47,4","48,1","48,7","48,7","48,0","47,8","47,6","46,4","46,3","46,3"
7,NaN,Second birth,"37,7","37,6","37,4","37,2","35,5","35,8","36,8","36,9",...,"37,5","37,3","36,7","36,6","37,2","37,6","37,6","39,0","38,6","39,1"
8,NaN,Third birth or higher,"15,6","15,8","15,2","15,0","14,4","14,4","14,8","15,4",...,"15,1","14,7","14,6","14,7","14,7","14,6","14,8","14,6",15,"14,6"
9,Estonia,First birth,"43,5","43,5","44,0","46,2","49,5","50,3","49,6","49,6",...,"41,9","42,3","40,8","40,2","36,7","38,8","38,0","37,2","39,8","39,7"


In [487]:
df_17_2 = pd.read_csv('../data/Raw/OECD/SF_2_1_Total_Fertility_rates_S1.csv')
df_17_2

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Australia,"3,45","3,55","3,43","3,34","3,15","2,97","2,89","2,85","2,89",...,"1,79","1,79","1,79","1,74","1,74","1,67","1,59","1,70","1,63","1,50"
1,Austria,"2,69","2,78","2,80","2,82","2,79","2,70","2,66","2,62","2,58",...,"1,46","1,49","1,53","1,52","1,48","1,46","1,44","1,48","1,41","1,32"
2,Belgium,"2,54","2,63","2,59","2,68","2,71","2,61","2,52","2,41","2,31",...,"1,74","1,70","1,68","1,65","1,62","1,60","1,55","1,60","1,53","1,47"
3,Canada,"3,90","3,84","3,76","3,67","3,50","3,15","2,81","2,60","2,45",...,"1,61","1,60","1,59","1,55","1,51","1,47","1,41","1,44","1,33","1,26"
4,Chile,"4,70","4,66","4,60","4,54","4,46","4,36","4,26","4,14","4,03",...,"1,77","1,74","1,69","1,56","1,54","1,43","1,31","1,18","1,26","1,17"
5,Colombia,"6,74","6,71","6,66","6,58","6,48","6,33","6,16","5,96","5,74",...,"1,82","1,77","1,72","1,72","1,72","1,71","1,69","1,68","1,66","1,65"
6,Costa Rica,"6,71","6,65","6,54","6,39","6,19","5,96","5,70","5,42","5,03",...,"1,80","1,80","1,75","1,71","1,71","1,60","1,45","1,35","1,34","1,33"
7,Czechia,"2,11","2,13","2,14","2,33","2,36","2,18","2,01","1,90","1,83",...,"1,53","1,57","1,63","1,69","1,71","1,71","1,71","1,83","1,62","1,45"
8,Denmark,"2,54","2,55","2,54","2,64","2,60","2,61","2,62","2,35","2,12",...,"1,69","1,71","1,79","1,75","1,73","1,70","1,67","1,72","1,55","1,50"
9,Estonia,"1,98","1,98","1,95","1,89","1,94","1,88","1,87","1,90","2,03",...,"1,54","1,58","1,60","1,59","1,67","1,66","1,58","1,61","1,41","1,31"


In [488]:
df_6666 = pd.read_csv('../data/Raw/OECD/OECD_df_famliy_selected.csv')
df_6666

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,COU,Country,SEX,Sex,IND,Indicator,...,OBS_VALUE,Observation Value,OBS_STATUS,Observation Status,UNIT_MEASURE,Unit of Measures,UNIT_MULT,Multiplier,BASE_PER,Base reference period
0,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,LVA,Latvia,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,39.5,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
1,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,GRC,Greece,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,11.1,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
2,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,CHL,Chile,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,74.8,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
3,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,NLD,Netherlands,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,51.9,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
4,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,LTU,Lithuania,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,26.4,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,COL,Colombia,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.4,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
501,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.5,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
502,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.6,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
503,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.7,NaN,A,NaN,YR,Years,0,Units,NaN,NaN


In [489]:
df_18 = pd.read_csv('../data/Raw/OECD/sf1_2_wide_from_df18.csv')
df_18

,country,year,Living with two parents,Living with a single parent,Other
0,Australia,2003,80.1,19.5,0.5
1,Australia,2006,81.5,18.0,0.5
2,Australia,2009,82.0,17.6,0.4
3,Australia,2012,81.3,18.0,0.6
4,Austria,2003,81.2,16.8,2.0
...,...,...,...,...,...
470,United States,2014,68.7,27.5,3.8
471,United States,2015,69.2,26.8,3.9
472,United States,2016,68.7,27.4,3.8
473,United States,2017,68.9,27.1,4.0


In [490]:
for col in df_18.select_dtypes(include=['object']).columns:
    df_18[col] = df_18[col].astype(str).str.strip()

# 2) Define placeholders representing missing data in OECD exports
placeholders = ['..', '...', '.', ' .', '…', 'Na', 'nan', 'None']

# 3) Replace placeholders with NaN directly in df_18
df_18.replace(placeholders, pd.NA, inplace=True)

In [491]:
# 1) Ensure 'year' is integer
df_18["year"] = pd.to_numeric(df_18["year"], errors="coerce").astype("Int64")

# 2) Convert all non-key columns to numeric and round(2)
for col in df_18.columns:
    if col not in ["country", "year"]:
        df_18[col] = pd.to_numeric(df_18[col], errors="coerce").round(2)

In [492]:
# 1) Drop rows with missing key fields
df_7777.dropna(subset=["country", "year"], inplace=True)

# 2) Drop duplicate country-year rows, keep the first
df_7777.drop_duplicates(subset=["country", "year"], keep="first", inplace=True)

# 3) Drop rows where all value columns are NaN
value_cols = [c for c in df_18.columns if c not in ["country", "year"]]
df_7777.dropna(subset=value_cols, how="all", inplace=True)

# 4) Sort and reset index
df_7777.sort_values(["country", "year"], inplace=True)
df_7777.reset_index(drop=True, inplace=True)


In [493]:
df_18

,country,year,Living with two parents,Living with a single parent,Other
0,Australia,2003,80.1,19.5,0.5
1,Australia,2006,81.5,18.0,0.5
2,Australia,2009,82.0,17.6,0.4
3,Australia,2012,81.3,18.0,0.6
4,Austria,2003,81.2,16.8,2.0
...,...,...,...,...,...
470,United States,2014,68.7,27.5,3.8
471,United States,2015,69.2,26.8,3.9
472,United States,2016,68.7,27.4,3.8
473,United States,2017,68.9,27.1,4.0


In [494]:
df_18['Other'] = pd.to_numeric(df_18['Other'], errors='coerce')

In [495]:
df_info = pd.DataFrame({
    'dtype': df_18.dtypes,
    'null_count': df_18.isnull().sum(),
    'unique_count': df_18.nunique()
})
print(df_info)

                               dtype  null_count  unique_count
country                       object           0            39
year                           Int64           0            18
Living with two parents      float64           0           211
Living with a single parent  float64           0           203
Other                        float64           1            50


In [496]:
print(repr(df_18.loc[df_18['Other'].notnull(), 'Other'].unique()))

array([0.5, 0.4, 0.6, 2. , 1. , 1.9, 0.3, 0.1, 0.8, 0.7, 8.7, 3.5, 2.5,
       2.1, 2.4, 2.6, 6.7, 5.1, 1.4, 1.2, 1.7, 1.5, 3.4, 2.9, 2.3, 3. ,
       4.2, 2.8, 1.3, 9. , 0.2, 0.9, 1.1, 4.5, 4.7, 1.6, 3.8, 3.6, 3.3,
       2.2, 0. , 1.8, 2.7, 3.2, 3.9, 4.1, 4.4, 3.7, 4. , 4.3])


In [497]:
df_18['Other'] = pd.to_numeric(df_18['Other'], errors='coerce')

df_18.dropna(inplace=True, subset=['Other'])

df_18.isnull().sum()

country                        0
year                           0
Living with two parents        0
Living with a single parent    0
Other                          0
dtype: int64

In [498]:
#df_18.to_csv('../data/Cleaned/cleaned_household_children.csv', index=False)

In [499]:
#df_18.to_sql('household_children_oecd', engine, if_exists= 'replace', index= False)

In [500]:
df_19_1 =pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_mean_age_birth_S1.csv')
df_19_1

,Country,1963,1964,1965,1966,1967,1968,1969,1970,1971,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,"27,5","27,5","27,4","27,3","27,3","27,2","27,2","27,1","26,9",...,"30,1","30,1","30,2","30,3","30,5","30,6","30,7","30,8","30,9","31,1"
1,Austria,"27,4","27,4","27,3","27,1","27,0","26,8","26,8","26,7","26,7",...,"30,2","30,3","30,4","30,6","30,6","30,7","30,9","31,0","31,0","31,2"
2,Belgium,"27,8","27,7","27,6","27,5","27,4","27,3","27,2","27,2","27,0",...,"30,0","30,2","30,3","30,4","30,5","30,6","30,7","30,8","30,8","31,0"
3,Canada,"27,8","27,9","27,8","27,7","27,5","27,3","27,3","27,2","27,0",...,"30,3","30,4","30,5","30,6","30,7","30,9","31,0","31,2","31,3","31,4"
4,Chile,"29,2","29,1","29,1","29,0","28,8","28,7","28,6","28,4","28,2",...,"28,1","28,3","28,5","28,8","29,1","29,4","29,6","29,9","30,1",..
5,Czech Republic,"25,7","25,8","25,5","25,2","25,0","24,9","24,8","24,8","24,9",...,"29,8","29,9","29,9","30,0","30,0","30,0","30,1","30,2","30,2","30,4"
6,Costa Rica,"29,3","29,3","29,3","29,3","29,2","29,1","28,9","28,7","28,5",...,"26,5","26,7","26,8","27,1","27,2","27,4","27,6","27,9","28,4","28,7"
7,Denmark,"27,3","26,8","26,8","26,6","26,5","26,5","26,6","26,7","26,7",...,"30,7","30,8","30,9","31,0","31,0","31,1","31,2","31,3","31,4","31,6"
8,Estonia,"27,6","27,4","27,3","27,3","27,1","26,9","26,9","26,7","26,7",...,"29,6","29,5","29,6","29,9","30,2","30,4","30,5","30,6","30,7","31,0"
9,Finland,"28,1","28,0","28,0","27,8","27,7","27,5","27,4","27,1","26,9",...,"30,4","30,5","30,5","30,6","30,8","30,9","31,0","31,1","31,2","31,4"


In [501]:
df_19_2 = pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_fertility_by_age_1960_S2.csv')
df_19_2

,Country,Age group,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,15-19,"44,3","47,4","44,7","45,9","47,0","47,5","48,9","48,4",...,"16,1","14,6","12,9","12,0","10,5","10,3","9,5","8,8","7,9","7,1"
1,Australia,20-24,"220,1","225,8","216,0","208,2","190,5","179,3","173,1","170,8",...,"53,2","51,3","47,4","47,3","44,7","43,1","42,8","40,1","37,7","38,8"
2,Australia,25-29,"216,3","221,2","216,7","211,2","198,1","188,5","183,9","185,0",...,"102,6","99,1","94,8","93,4","92,2","89,7","89,3","84,3","80,3","86,7"
3,Australia,30-34,"127,5","131,1","127,7","123,9","119,1","110,1","105,1","102,8",...,"126,9","124,8","120,4","121,7","123,6","119,1","120,1","115,6","111,4","120,6"
4,Australia,35-39,"62,3","63,4","61,4","59,7","58,4","53,0","50,6","47,8",...,"71,5","70,9","69,2","69,8","72,0","71,3","71,6","69,3","66,3","70,9"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Bulgaria,25-29,"120,1","119,3","114,8","113,5","107,7","104,5","102,1","101,3",...,"88,9","86,7","88,1","88,1","88,5","88,6","88,8","91,3","89,8","91,7"
143,Bulgaria,30-34,"51,3","52,1","51,7","49,9","48,2","45,1","41,8","40,3",...,"65,8","64,6","68,5","69,7","71,2","71,5","72,3","72,8","71,3","73,6"
144,Bulgaria,35-39,"19,7","19,4","19,2","18,1","16,9","16,7","15,3","14,6",...,"27,6","27,1","30,1","30,6","31,2","32,7","32,9","34,0","34,0","34,8"
145,Bulgaria,40-44,"7,2","6,4","6,3","5,4","5,0","4,4","3,8","3,8",...,"4,4","4,7","5,4","5,9","6,6","7,1","7,2","7,7","7,8","7,8"


In [502]:
df_19_3 = pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_fertility_by_age_2000_S3.csv')
df_19_3

,Country,Age group,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,OECD-Average,15-19,"22,6","22,0","21,1","20,5","20,3","20,1","20,0","20,5",...,"17,9","16,8","16,2","15,2","14,4","13,5","12,6","11,7","10,2","9,5"
1,OECD-Average,20-24,"71,7","69,3","66,8","65,5","64,7","63,2","62,9","63,0",...,"56,4","53,8","53,3","51,9","50,4","48,8","47,0","45,0","42,0","40,5"
2,OECD-Average,25-29,"107,9","105,0","103,1","103,5","103,4","102,3","102,6","103,4",...,"99,4","96,5","96,9","96,1","94,9","92,8","90,7","88,4","85,5","86,9"
3,OECD-Average,30-34,"88,1","87,2","88,6","91,1","93,4","94,6","97,6","100,0",...,"103,6","101,9","104,0","104,9","105,3","104,1","103,3","101,7","99,6","103,6"
4,OECD-Average,35-39,"38,1","38,6","39,5","40,6","42,2","43,5","45,6","47,7",...,"53,1","53,4","55,1","56,3","57,1","57,0","57,4","57,5","55,9","58,7"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,Romania,25-29,"78,2","77,0","78,6","82,0","84,8","90,8","92,3","93,0",...,"91,8","88,3","94,4","98,9","100,1","109,0","108,3","109,1","109,4","110,9"
297,Romania,30-34,"38,8","38,1","38,8","38,8","41,6","47,5","51,1","54,2",...,"66,6","64,8","71,5","75,4","78,5","86,6","85,9","86,4","87,1","87,5"
298,Romania,35-39,"13,4","13,8","15,2","19,4","23,2","25,1","25,7","24,9",...,"27,3","27,4","29,9","32,1","33,0","36,8","36,7","38,3","40,6","41,1"
299,Romania,40-44,"3,1","3,1","3,0","2,9","3,1","3,1","2,8","3,1",...,"4,9","4,8","5,6","6,1","6,8","7,3","7,8","8,0","8,5","8,2"


In [503]:
df_888= pd.read_csv('../data/Raw/OECD/Households-by-type,-presence-of-children-and-country,-2015-2024.csv')
df_888

,Category,Single adult with children,Single adult without children,Couple with children,Couple without children,Other type of household with children,Other type of household without children
0,2015,6147.3,64181.3,31679.8,46641.6,11698.9,30771.6
1,2016,6148.5,63891.1,31907.3,47308.2,11766.3,30559.5
2,2017,6108.5,65353.9,32091.5,47426.1,11530.2,30297.5
3,2018,6163.6,66165.5,31720.2,48194.8,11342.5,30224.0
4,2019,6246.4,67417.9,31710.1,48503.6,11285.7,30134.8
5,2020,6136.4,67412.9,31622.2,48831.2,11212.9,30445.2
6,2021,5691.9,70200.4,30558.3,47447.4,11611.8,30700.7
7,2022,5984.9,72134.3,30469.3,47995.5,11513.6,30412.1
8,2023,5924.8,73396.2,30313.0,48477.5,11443.5,30608.8
9,2024,6077.7,75049.7,30286.5,49058.4,11311.9,30487.3


In [506]:
df_20= pd.read_csv('../data/Raw/OECD/SF_2_4_Share_births_outside_marriage_1960.csv')
df_20

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Austria,"13,0","12,6","12,0","11,6","11,3","11,2","11,4","11,5","12,0",...,"40,4","41,5","41,4","41,7","42,1","42,2","42,0","41,3","40,6","41,2"
1,Belgium,"2,1","2,0","2,1","2,2","2,3","2,4","2,5","2,5","2,7",...,"47,0","47,7","49,5","49,4","48,0","49,0","52,8","52,4",..,..
2,Czech Republic,"4,9","4,6","4,5","4,7","4,8","5,0","5,3","5,3","5,4",...,"41,8","43,4","45,0","46,7","47,8","48,6","49,0","48,5","48,2","48,5"
3,Denmark,"7,8","8,0","8,3","8,9","9,3","9,5","10,2","11,1","11,1",...,"49,0","50,6","51,5","52,5","53,8","54,0","54,2","54,2","54,1","54,2"
4,Finland,"4,0","4,1","4,0","4,2","4,4","4,6","4,8","5,1","5,3",...,"40,9","41,5","42,1","42,8","44,3","44,9","44,8","44,6","45,4","46,1"
5,Germany,"7,6","7,1","6,6","6,1","5,9","5,8","5,7","5,8","6,1",...,"33,9","34,5","34,8","35,0","35,0","35,5","34,7","33,9","33,3","33,1"
6,Greece,"1,2","1,2","1,2","1,2","1,1","1,1","1,0","1,0","1,1",...,"7,4","7,6","7,0","8,2","8,8","9,4","10,3","11,1","12,4","13,8"
7,Hungary,"5,5","5,5","5,4","5,3","5,2","5,2","5,1","5,0","5,0",...,"42,3","44,5","45,6","47,3","47,9","46,7","44,7","43,9","38,7","30,4"
8,Iceland,"25,3","25,3","24,5","25,1","26,7","26,9","28,4","30,0","30,5",...,"65,0","66,9",..,..,..,"69,6","71,2","70,5","69,4",..
9,Ireland,"1,6","1,6","1,8","1,8","2,0","2,2","2,3","2,5","2,6",...,"33,9","35,1","35,3","36,3","36,6","36,7","37,6","37,9","38,4",..


In [507]:
df_21_1= pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rate_mean_age_first_marriage_S1.csv')
df_21_1

,Country,Gender,1990,1991,1992,1993,1994,1995,1996,1997,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Australia,Male,"26,5","26,7","26,9","27,0","27,2","27,3","27,6","27,8",...,"29,7","29,8","29,9","30,0","30,1","30,3","30,4","30,7","30,7","30,6"
1,Australia,Female,"24,3","24,5","24,7","24,8","25,1","25,3","25,7","25,9",...,"28,0","28,1","28,3","28,4","28,5","28,7","28,8","29,2","29,3","29,2"
2,Czechia,Male,"24,3","24,3","24,5","24,7","25,1","25,5","25,9","26,5",...,"31,0","31,2","31,3","31,4","31,6","31,7","31,8","31,9","32,0","32,4"
3,Czechia,Female,"21,6","21,6","21,9","22,1","22,4","22,8","23,1","23,6",...,"28,1","28,3","28,5","28,7","28,8","29,0","29,1","29,2","29,4","29,7"
4,Denmark,Male,"30,5","30,6","31,0","31,4","31,8","31,9","32,5","32,2",...,"33,8","34,3","34,4","34,4","34,3","34,7","34,8","34,9","35,1","35,3"
5,Denmark,Female,"27,8","28,0","28,3","28,8","29,2","29,2","29,9","30,1",...,"31,4","31,8","31,9","31,9","31,9","32,2","32,4","32,5","32,8","33,0"
6,Greece,Male,"29,0","29,3","29,6","29,7","29,9","30,1","30,2","30,6",...,"32,7","32,8","32,9","33,0","33,2","33,2","33,3","33,4","33,7","33,8"
7,Greece,Female,"24,9","25,2","25,5","25,5","25,8","26,0","26,3","26,6",...,"29,4","29,5","29,7","29,9","30,1","30,1","30,3","30,3","30,7","30,7"
8,Japan,Male,"28,4","28,4","28,4","28,4","28,5","28,5","28,5","28,5",...,"30,7","30,8","30,9","31,1","31,1","31,1","31,1","31,1","31,2","31,0"
9,Japan,Female,"25,9","25,9","26,0","26,1","26,2","26,3","26,4","26,6",...,"29,0","29,2","29,3","29,4","29,4","29,4","29,4","29,4","29,6","29,4"


In [508]:
df_21_2 = pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rates_S2.csv')
df_21_2

,Country,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Austria,"1,4","1,3","1,3","1,3","1,4","1,4","1,5","1,5","1,6",...,"1,9","1,9","1,9","1,8","1,8","1,8","1,8","1,7","1,6","1,5"
1,Belgium,"0,7","0,7","0,8","0,9","1,0","1,1","1,3","1,3","1,4",...,"2,2","2,2","2,2","2,1","2,0","2,0","2,0","1,8","1,9","1,7"
2,Czechia,"2,2","2,4","2,3","2,5","2,5","2,6","2,5","2,5","2,6",...,"2,7","2,5","2,5","2,4","2,4","2,3","2,3","2,0","2,0","1,9"
3,Denmark,"1,9","2,7","2,6","2,5","2,6","2,6","2,6","2,6","2,6",...,"3,4","3,4","2,9","3,0","2,6","2,6","1,8","2,7","2,2","2,1"
4,Estonia,"3,2","3,2","3,3","3,2","3,3","3,4","3,6","3,9","3,8",...,"2,5","2,4","2,6","2,5","2,5","2,4","2,1","1,9",NaN,"1,9"
5,Finland,"1,3","1,6","1,8","1,9","2,1","2,0","2,1","2,1","2,2",...,"2,5","2,5","2,5","2,5","2,4","2,4","2,4","2,4","2,2","2,0"
6,Germany,"1,3","1,4","1,5","1,6","1,8","1,9","2,0","1,5","1,0",...,"2,1","2,1","2,0","2,0","1,9","1,8","1,8","1,7","1,7","1,6"
7,Greece,"0,4","0,4","0,4","0,5","0,4","0,4","0,4","0,5","0,5",...,"1,5","1,3","1,4","1,0","1,8",NaN,NaN,NaN,NaN,NaN
8,Hungary,"2,2","2,3","2,3","2,4","2,3","2,5","2,6","2,6","2,7",...,"2,0","2,0","2,1","2,0","1,9","1,7","1,8","1,5","1,9","1,8"
9,Italy,..,"0,3","0,6","0,3","0,3","0,2","0,2","0,2","0,2",...,"0,9","0,9","1,4","1,6","1,5","1,5","1,4","1,1","1,4","1,4"


In [512]:
df_21_3= pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rates_prev_marital_status_S3.csv')
df_21_3

,Country,Previous marital status,2000,2001,2002,2003,2004,2005,2006,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Australia,Single never married,"75,9","76,1","75,5","75,6","76,2","76,9","77,3","78,2",...,"79,6","79,7","80,0","80,5","80,5","80,1","80,3","80,1","80,3","80,7"
1,Australia,Divorced,"22,0","21,8","22,4","22,3","21,8","21,3","20,9","20,2",...,"19,0","18,8","18,6","18,2","18,1","18,5","18,3","18,5","18,3","18,0"
2,Australia,Widowed,"2,1","2,1","2,1","2,1","1,9","1,8","1,8","1,7",...,"1,5","1,5","1,4","1,3","1,4","1,4","1,4","1,4","1,3","1,3"
3,Austria,Single never married,"76,6","74,7","74,1","73,7","72,9","73,1","73,9","74,8",...,"75,5","75,7","76,7","77,1","77,5","77,7","78,1","78,1","78,2","78,0"
4,Austria,Divorced,"22,2","24,2","24,7","25,2","25,9","25,7","24,9","24,2",...,"23,5","23,4","22,3","22,0","21,5","21,5","20,9","21,0","21,0","21,6"
5,Austria,Widowed,"1,2","1,1","1,2","1,1","1,2","1,2","1,1","1,0",...,"1,0","0,9","1,0","0,9","1,0","0,8","0,9","0,9","0,8","0,4"
6,Czechia,Single never married,"74,9","74,5","74,3","74,0","73,9","74,2","74,5","72,6",...,"74,0","74,0","75,2","75,6","76,6","76,7","76,4","76,4","76,1","75,9"
7,Czechia,Divorced,"23,7","24,2","24,4","24,7","24,7","24,5","24,4","26,1",...,"24,9","24,9","23,8","23,4","22,4","22,3","22,6","22,6","22,9","23,0"
8,Czechia,Widowed,"1,4","1,3","1,3","1,3","1,4","1,2","1,1","1,3",...,"1,2","1,1","1,0","1,0","1,0","1,0","1,0","1,0","1,0","1,1"
9,Denmark,Single never married,"75,9","76,0","76,2","76,4","76,0","75,6","75,6","76,3",...,"77,2","76,0","75,0","76,2","76,1","76,9","76,4","77,1","77,6","78,3"


In [515]:
df_22_1 = pd.read_csv('../data/Raw/OECD/SF3_3_A_in_private_households_by_partnership_status_S1.csv')
df_22_1

,Country,20+_All_Total_Living_with_a_partner,20+_All_Married or in a civil or registered partnership_living_with_a_partner,20+_All_Cohabiting_living_with_a_partner,20+_All_Not living with a partner,Unnamed: 5,20/34_Total_living_with_a_partner,20/34_Married or in a civil or registered partnership_living_with_a_partner,20/34_Cohabiting_living_with_a_partner,Not living with a partner_Total,Living with at least one parent
0,Australia (c),"63,79","53,59","10,20","36,21",NaN,"47,06","29,41","17,65","52,94",..
1,Austria,"58,80","49,10","9,70","41,20",NaN,"39,11","22,15","16,97","60,89","33,82"
2,Belgium,"62,15","53,51","8,64","37,85",NaN,"45,28","29,33","15,94","54,72","31,34"
3,Canada (d),"66,89","54,46","12,43","33,11",NaN,"55,34","33,55","21,79","44,66",..
4,Czech Republic,"51,17","45,39","5,79","48,83",NaN,"30,78","21,32","9,46","69,22","36,20"
5,Denmark,"64,15","50,02","14,12","35,85",NaN,"50,54","21,86","28,68","49,46","10,67"
6,Estonia,"53,93","37,30","16,64","46,07",NaN,"45,31","17,81","27,50","54,69","26,46"
7,France,"64,14","49,41","14,72","35,86",NaN,"50,42","21,89","28,53","49,58","22,08"
8,Germany,"62,61","53,91","8,69","37,39",NaN,"39,53","22,15","17,39","59,74","27,54"
9,Greece,"60,23","58,52","1,71","39,77",NaN,"33,13","29,24","3,90","66,87","45,43"


In [516]:
df_22_2 = pd.read_csv('../data/Raw/OECD/SF3_3_B_ by level of educational attainment_S2.csv')
df_22_2

,Country,Low_Education_Total_living_with_a_partner,Low_educationMarried or in a civil or registered partnership_living_with_a_partner,Low_education_Cohabiting_living_with_a_partner,Not living with a partner_Low_education,Medium education_Total_Living with a partner,Medium education_Married or in a civil or registered partnership_Living with a partner,Medium education_Cohabiting_Living with a partner,Not living with a partner_Medium education,Unnamed: 9,High education_Total_Living with a partner,High education_Married or in a civil or registered partnership_Living with a partner,High education_Cohabiting_Living with a partner,Not living with a partner_High education_
0,Austria,"56,81","50,49","6,32","43,19","59,27","48,73","10,54",NaN,NaN,"60,03","48,38","11,65","39,97"
1,Belgium,"62,28","56,11","6,17","37,72","60,79","49,80","10,99",NaN,NaN,"67,09","56,58","10,51","32,91"
2,Czech Republic,"40,81","36,55","4,26","59,19","53,99","47,87","6,12","46,01",NaN,"57,29","50,26","7,03","42,71"
3,Estonia,"42,17","26,39","15,78","57,83","54,41","36,61","17,79","45,59",NaN,"60,14","44,45","15,69","39,86"
4,France,"61,12","51,93","9,18","38,88","65,68","49,17","16,51","34,32",NaN,"65,58","46,60","18,98","34,42"
5,Germany,"54,46","48,79","5,67","45,54","62,38","53,13","9,25","37,62",NaN,"68,89","59,16","9,74","31,11"
6,Greece,"63,81","62,88","0,93","36,19","57,00","54,88","2,12","43,00",NaN,"58,33","55,70","2,63","41,67"
7,Hungary,"50,33","40,38","9,95","49,67","57,94","46,78","11,15","42,06",NaN,"59,56","51,02","8,55","40,44"
8,Iceland,"51,86","41,02","10,84","48,14","58,31","46,57","11,74","41,69",NaN,"69,72","54,53","15,19","30,28"
9,Latvia,"36,27","25,92","10,35","63,73","49,32","39,54","9,78","50,68",NaN,"52,91","45,39","7,52","47,09"


In [504]:
df_999 = pd.read_csv('../data/Raw/OECD/Households-with-children-by-number-of-children,-2024.csv')
df_999

,Category,1 child,2 children,3 children or more
0,European Union,11.7,8.9,3.0
1,NaN,NaN,NaN,NaN
2,Slovakia,17.1,14.5,4.0
3,Ireland,12.4,12.2,6.4
4,Cyprus,13.9,11.7,3.1
5,Czechia,13.9,11.6,2.6
6,Romania,14.3,9.2,4.0
7,Luxembourg,12.5,12.1,2.4
8,Belgium,11.8,10.2,4.1
9,Croatia,12.0,10.1,3.8


In [505]:
import pandas as pd

df = pd.read_csv('../data/Raw/OECD/OECD,DF_FAMILY,+all.csv')

df_wide = df.pivot_table(
    index=['Country', 'TIME_PERIOD', 'COU'],
    columns='Indicator',
    values='OBS_VALUE'
).reset_index()

df_wide.columns.name = None

df_wide.to_csv("WIDE_FORMAT.csv", index=False)

print(df_wide)

# df_wide = df.pivot(index='id', columns='variable', values='value')
# df = df.drop(columns='indicator')

            Country  TIME_PERIOD  COU  Child poverty rate  \
0         Argentina         2001  ARG                 NaN   
1         Argentina         2002  ARG                 NaN   
2         Argentina         2003  ARG                 NaN   
3         Argentina         2004  ARG                 NaN   
4         Argentina         2005  ARG                 NaN   
...             ...          ...  ...                 ...   
1170  United States         2018  USA                 NaN   
1171  United States         2019  USA                 NaN   
1172  United States         2020  USA                 NaN   
1173  United States         2021  USA                 NaN   
1174  United States         2022  USA                 NaN   

      Country mean average score in mathematics, by sex  \
0                                                   NaN   
1                                                   NaN   
2                                                   NaN   
3                              